<a href="https://colab.research.google.com/github/hungpham13/Laptop-EDA/blob/main/DS_Project_20211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hungpham13/Laptop-EDA.git

Cloning into 'Laptop-EDA'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 229 (delta 138), reused 113 (delta 52), pack-reused 0
Receiving objects: 100% (229/229), 24.43 MiB | 10.22 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Checking out files: 100% (25/25), done.


**[Guideline](https://husteduvn.sharepoint.com/sites/IntroductiontoDataScience-IT4142E-2021-2022/Class%20Materials/Capstone%20project/DS-Capstone%20project%20guidelines.pdf)**
## Topic description assignment (20th of October, 2021)
[Our project proposal](https://docs.google.com/document/d/16L1Mqy_HtDfhxB0xNXmCZSBNMKazWsYgwA9s1Vueih8/edit?usp=sharing)


## Assginment for each member
### **[On GG Sheet](https://docs.google.com/spreadsheets/d/11ldCYnPJwYuMKBba6jX7qPYnYtekAYjR9YvORww76hE/edit#)**

Distinguish between programming tasks and analytic tasks. 
- Examples of programming tasks: data scraping for source A; implementing algorithm A,
etc.
- Examples of analytic tasks: proposing our subject, selecting the algorithms to be used to
solve this problem, writing parts 1/2/3/4 of the report, writing parts 1/2/3/4 of the presentation, etc.

For each task, provide a detailed contribution of each member to this task:
- e.g.: Programming Task 1- data scraping from source A: member1 Mr X 40% ; member 2 Mrs Y 60% (no more than 2 members /task for programming tasks)...).
- e.g.: Analytic Task 1– proposing our subject: member1 Mr X 40% ; member 2 Mrs Y 30% ; member 2 Mr Z 30% (can be more than 2 members /task for analytic
tasks)...)

## Final assignment (29th of December, 2021)
    
Create 1 directory named with group number, upload on the dedicated assignment on Teams:
- Report: PDF file, no more than 15 pages (not included reference pages / first page / table of contents)
- Code: put in a single .zip file, with a readme.txt file explaining your codes structure, how to run the project – if necessary.
- Video demo of your Python code running in a notebook
- Presentation: PDF file, no more than 10 slides. The first slide MUST contain the (recent) pictures of the students in the group, with student’s name/number

# Materials

1.Scrapy, Selenium, BeautifulSoup –Data scraping
2.Numpy, Pandas, Seaborn –data analysis and visualization

[Kaggle detailed EDA](https://www.kaggle.com/ekami66/detailed-exploratory-data-analysis-with-python)

[Github DS project](https://github.com/veb-101/Data-Science-Projects)

[Havard Data Science Course](http://cs109.github.io/2015/pages/videos.html)

[Web Scraping with Python (Book)](https://vn1lib.org/book/3515980/5d50aa)

[Web Scraping on Kaggle](https://www.kaggle.com/getting-started?search=web+scraping)

In [2]:
import re
import pandas as pd
import numpy as np
import glob

!pip install python-Levenshtein
!pip install fuzzywuzzy
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz

     |████████████████████████████████| 50 kB 2.6 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=85b5a5ad75d006bae0cc2fa4566db686b1bf1c921f4e7b96f62f5ed898788c79
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


# Data cleaning - Data preprocessing

In [3]:
def text_facet(df, col, print_all=True,ascending=False):
    if print_all:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(f"---------- {col} ------------")
            print(df[col].value_counts(dropna=False,ascending=ascending))
    else:
        print(f"---------- {col} ------------")
        print(df[col].value_counts(dropna=False,ascending=ascending))
        print(f"Number of null: {df[col].isna().sum()}")


def lower_all_string(df):
    '''lower all values in string columns of dataframe df'''
    for c in df.columns:
        if df[c].dtypes == np.object:
            df[c] = df[c].str.lower()

data_path = 'Laptop-EDA/scraped_data/'
all_brands = ['apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft',
              'huawei','avita','gigabyte','fujitsu']

In [4]:
retained_cols = ['Comment_count','Price', 'Rating', 'Rating_count']
mapping_cols = ['Series','Brand','cpu_brand','cpu_code','ram_size',\
                'storage_size','displ_size','displ_rate']
other_cols = ['Battery', 'GPU', 'OS', 'P/N',  'displ_resolution', 'hdmi',\
            'Release_year', 'Surface', 'Weight', 'backlit_keyboard', \
            'lan', 'length', 'ram_upgradable', 'screen_technology',\
            'sd_card', 'storage_extra_slot', 'storage_type', \
            'headphone_jack', 'thickness', 'type_c', 'usb_a', 'width']
all_cols = retained_cols + mapping_cols + other_cols

## Thegioididong - Dienmayxanh
result: no screen technology

In [5]:
def clean_thegioididong(source):
    source.dropna(subset=['ports'],inplace=True)
    result = pd.DataFrame()
    result['P/N'] = source['title'].str.extract(r"\(([^)]+)\)",expand=False).str.strip().str.replace("\.$", "") #replace any dot at the end of P/N

    result['Rating'] = source['rating'].fillna(0).astype('float64')
    result['Rating_count'] = source['review_count'].str.extract(r'(\d+)',expand=False).str.strip().fillna(0).astype('int64')

    result['Comment_count'] = 0
    for i,r in source['cmt_count'].iteritems():
        if pd.isnull(r):
            continue
        for c in r.split("|"):
            if re.search(r'([\d.,]+)',c):
                result['Comment_count'][i] += int(re.search(r'([\d.,]+)',c).group().replace(".",""))
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['Comment_count'].value_counts(dropna=False))

    result['Brand'] = source['title'].str.split().str[1].str.strip().str.lower()
    # print(result['Brand'].value_counts(dropna=False))
    result['Series'] = source['title'].str.extract(r'\w+\s\w+\s([\w\s]+)\s[\w\/d]+',expand=False).str.strip()
    result.loc[(result['Series'].str.split().str[-1].str.len() == 2), 'Series'] =  result['Series'].str.slice(stop=-3)

    #add P/N to Series (except Apple laptop)
    result['Series'] = np.where(result['Brand'] != 'apple', 
                                result[['Series', 'P/N']].fillna('').agg(' '.join, axis=1),
                                result['Series'])
    for a in zip(result.Series, source.title):
        print('{0:40} {1}'.format(a[0],a[1]))

    result['Price'] = source['price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')

    result['cpu_brand'] = source['cpu_name'].str.extract(r'([\w]+)\s',expand=False).str.strip()
    result['cpu_code'] = source['cpu_name'].str.extract(r'-([\w\d\s]+)',expand=False).str.strip()
    result['cpu_code'] = np.where(result['cpu_code'].str.len() > 10, 
                                    source['cpu_name'].str.extract(r'([\w\d\s]+)-',expand=False).str.split().str.slice(start=1).str.join(' '),
                                    result['cpu_code'])

    result['ram_size'] = source['ram_size'].str.extract(r'(\d+)',expand=False).str.strip().fillna(8).astype('float64')
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['ram_size'].value_counts(dropna=False))

    result['ram_upgradable'] = np.where(source['ram_upgradable'].str.contains("Kh"), "no", "yes")

    result['GPU'] = source['gpu'].str.extract(r'-([\w\d\s-]+),?',expand=False).str.strip()

    result['Battery'] = source['battery'].str.extract(r'([\d\s.]+)Wh',expand=False).str.strip().astype('float64')

    result['storage_size'] = np.nan
    result['storage_type'] = np.nan
    result['storage_extra_slot'] = np.nan
    # for i,r in result.loc[result['storage_size'].notnull(),'storage_size'].iteritems():
    for i,r in source['storage'].iteritems():
        if pd.isnull(r):
            continue
        for s in r.split("|"):
            s = s.lower()
            if "hỗ trợ" not in s:
                result['storage_size'][i] = re.search(r'(\d+)\s?[gtb]{2}',s.strip()).group(1)
                if "ssd" in s:
                    result['storage_type'][i] = "SSD"
                elif "hdd" in s:
                    result['storage_type'][i] = "HDD"
            elif "không" not in s: #just have "ho tro", no "khong"
                result['storage_extra_slot'][i] = 'yes'
        if pd.isnull(result['storage_extra_slot'][i]):
                result['storage_extra_slot'][i] = 'no'
    result['storage_size'] = np.where(result['storage_size']==1,1024,result['storage_size'])
    result['storage_size'] = np.where(result['storage_size']==2,2048,result['storage_size'])
    # print(result['storage_size'].value_counts(dropna=False))

    result['displ_size'] = source['displ_size'].str.extract(r'([\d.]+)',expand=False).astype('float64')
    result['displ_resolution'] = source['displ_resolution'].str.extract(r"\(([^)]+)\)",expand=False).str.strip()
    result['displ_rate'] = source['displ_rate'].str.extract(r'([\d]+)',expand=False).fillna(60).astype('int64')
    result['screen_technology'] = np.nan
    for i,r in source.displ_tech.iteritems():
        if pd.isnull(r):
            continue
        elif 'ips' in r.lower():
            result['screen_technology'][i] = 'IPS'
        elif 'oled' in r.lower():
            result['screen_technology'][i] = 'OLED'
        elif 'wva' in r.lower():
            result['screen_technology'][i] = 'VA'
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['screen_technology'].value_counts(dropna=False))
    #     print(source.displ_tech[result.screen_technology.isnull()])

    result['Weight'] = source['weight_dims'].str.extract(r"Nặng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['length'] = source['weight_dims'].str.extract(r"Dài ([\d.]+)",expand=False).str.strip().astype('float64')
    result['width'] = source['weight_dims'].str.extract(r"Rộng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['thickness'] = source['weight_dims'].str.extract(r"Dày ([\d.]+)",expand=False).str.strip().astype('float64')

    result['OS'] = source['os'].str.split().str[:2].str.join(" ").str.strip()
    result['Surface'] = np.nan
    for i,r in source['surface'].iteritems():
        if pd.isnull(r):
            continue
        d = {"nhôm":"Aluminum","carbon":"Carbon","kim loại": "Metal", "nhựa":"Plastic","magie":"Magnesium"}
        s = []
        for vn_term in d:
            if vn_term in r.lower():
                s.append(d[vn_term])
        result['Surface'][i] = ', '.join(s)
        
    result['Release_year'] = source['launch']
    result['backlit_keyboard'] = np.where(source['backlight_key'].str.contains('Không'),'no','yes')

    result['hdmi'] = np.where(source['ports'].str.lower().str.contains("hdmi"),'yes','no')
    result['headphone_jack'] = np.where(source['ports'].str.lower().str.contains("jack"),'yes','no')
    result['lan'] = np.where(source['ports'].str.lower().str.contains("lan"),'yes','no')
    result['type_c'] = np.where(source['ports'].str.lower().str.contains("-c"),'yes','no')
    result['usb_a'] = np.where(source['ports'].str.lower().str.contains(r"usb [a32]",regex=True),'yes','no')
    result['sd_card'] = np.where(pd.isnull(source['sd_card']),'no','yes')
    lower_all_string(result)
    return result

In [6]:
scraped_tgdd = pd.read_csv(data_path+"tgdd_dmx.csv",index_col='url')
cleaned_tgdd = clean_thegioididong(scraped_tgdd)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(cleaned_tgdd.Series.value_counts(dropna=False))
#     print(cleaned_tgdd)
cleaned_tgdd.to_csv(data_path+'tgdd_dmx_cleaned.csv',encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: Set

Katana Gaming GF66 11UC 224VN            Laptop MSI Katana Gaming GF66 11UC i7 11800H/8GB/512GB/4GB RTX3050/144Hz/Balo/Win10 (224VN)
Gaming Pulse GL66 11UDK 816VN            Laptop MSI Gaming Pulse GL66 11UDK i7 11800H/16GB/512GB/4GB RTX3050Ti/144Hz/Balo/Chuột/Win10 (816VN)
Nitro 5 Gaming AN515 57 727J NH.QD9SV.005 Laptop Acer Nitro 5 Gaming AN515 57 727J i7 11800H/8GB/512GB/4GB RTX3050Ti/144Hz/Win10 (NH.QD9SV.005.)
Nitro 5 Gaming AN515 57 5831 NH.QDGSV.003 Laptop Acer Nitro 5 Gaming AN515 57 5831 i5 11400H/8GB/512GB/6GB RTX3060/144Hz/Win10 (NH.QDGSV.003)
Aspire 7 Gaming A715 42G R6ZR NH.QAYSV.003 Laptop Acer Aspire 7 Gaming A715 42G R6ZR R5 5500U/8GB/512GB/4GB GTX1650/144Hz/Win10 (NH.QAYSV.003)
Nitro 5 Gaming AN515 57 720A NH.QEQSV.004 Laptop Acer Nitro 5 Gaming AN515 57 720A i7 11800H/8GB/512GB/4GB RTX3050Ti/144Hz/Win11 (NH.QEQSV.004)
Gaming G5 KC-5S11130SB                   Laptop Gigabyte Gaming G5 i5 10500H/16GB/512GB/6GB RTX3060/144Hz/Win11 (KC-5S11130SB)
Nitro 5 Gaming AN515 57 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82:

## Fpt shop
- Rename column to the same format as data form
- Clean `Series`,`displ_rate`
- `Rating`,`Rating_count`: (5,1) to (0,0)

In [7]:
scraped_fpt = pd.read_csv(data_path+"fptshop_scraped.csv")
cleaned_fpt = pd.read_csv(data_path+"fpt-cleaned.csv")
cleaned_fpt.rename(columns={'PN':'P/N',
                            'rating':'Rating',
                            'rating_count':'Rating_count',
                            'comment_count':'Comment_count',
                            'price':'Price',
                            'RAM':'ram_size',
                            'RAM_upgradable':'ram_upgradable',
                            'battery_wh':'Battery',
                            'storage_upgradable':'storage_extra_slot',
                            'screen_size':'displ_size',
                            'display_resolution':'displ_resolution',
                            'display_rating':'displ_rate',
                            'weight':'Weight',
                            'height':'thickness',
                            'surface_material':'Surface',
                            'release_year':'Release_year', 
                            'jack':'headphone_jack',
                            'usb':'usb_a', 
                            'sd':'sd_card',
                            'keyboard_light':'backlit_keyboard'
                            },inplace=True)

lower_all_string(cleaned_fpt)
cleaned_fpt['P/N'] = cleaned_fpt['P/N'].fillna('').astype('object')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_fpt.info()
#     cleaned_fpt.info()
#     print(cleaned_fpt['Series'].value_counts(dropna=False))
for i,row in cleaned_fpt.iterrows():
    r = row.Series
    extract_r1 = re.search(r'([\w\s-]+)[ /]c?o?r?e?\s?[ir\d]{2}[- ]',r)
    extract_r2 = re.search(r'([\w\s-]+)pentium',r)
    extract_r3 = re.search(r'([\w\s-]+)celeron',r)
    extract_r4 = re.search(r'([\w\s-]+)n4020',r)
    extract_r5 = re.search(r'([\w\s-]+)/n4500',r)
    if extract_r1: extract_r = extract_r1.group(1).strip()
    elif extract_r2: extract_r = extract_r2.group(1).strip()
    elif extract_r3: extract_r = extract_r3.group(1).strip()
    elif extract_r4: extract_r = extract_r4.group(1).strip()
    elif extract_r5: extract_r = extract_r5.group(1).strip()
    else: extract_r = r.lower().strip()

    specs_key = [r'\d+gb',r'[\d.]+ghz',r'\d+gpu',r'\d+tb','core','m1']
    for k in specs_key:
        extract_r = re.sub(k, '', extract_r).strip(' -/')
        extract_r = ' '.join(extract_r.split())
    #add P/N
    if row['P/N'] not in extract_r and row.Brand != 'apple':
        extract_r = f"{extract_r} {row['P/N']}"

    print(f"{r:55}  {row['P/N']:20}  {extract_r}")
    cleaned_fpt.Series[i] = extract_r

#disp_rate
cleaned_fpt.displ_rate = cleaned_fpt.displ_rate.replace("--",60).fillna(60).astype('int64')
print(cleaned_fpt.displ_rate.value_counts(dropna=False))

#rating
cleaned_fpt.Rating[cleaned_fpt.Rating_count == 1] = 0
cleaned_fpt.Rating_count[cleaned_fpt.Rating_count == 1] = 0
cleaned_fpt.Rating_count = cleaned_fpt.Rating_count.fillna(0).astype('int64')

# cleaned_fpt[cleaned_fpt.Series == 'Flip BR1100FKA-BP0531T/N4500']

nitro gaming an515-57-74nu/i7-11800h                     nh.qd9sv.001          nitro gaming an515-57-74nu nh.qd9sv.001
modern 14 b11mou 852vn i5 1155g7/852vn                   852vn                 modern 14 b11mou 852vn
inspiron n3511 i3 1115g4/p112f001abl                     p112f001              inspiron n3511 p112f001
vostro v3500 i3 1115g4                                   v5i3001w              vostro v3500 v5i3001w
ideapad slim 3 15itl6 i3 1115g4                          82h80040vn            ideapad slim 3 15itl6 82h80040vn
inspiron n3511 i5 1135g7/p112f001bbl                     p112f001              inspiron n3511 p112f001
ideapad slim 5 15itl05 i5 1135g7                         82fg00r0vn            ideapad slim 5 15itl05 82fg00r0vn
gaming gf63 thin 10sc-481vn i7 10750h                    481vn                 gaming gf63 thin 10sc-481vn
inspiron n5515 r5 5500u/p106f003asl                      p106f003asl           inspiron n5515 p106f003asl
thinkbook 14 g3 acl r7 5700u/21a20

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Cellphone S
- Drop column `'1star', '2star', '3star', '4star', '5star'`
- Rename column `comment_count`
- Change Price to float
- recleaned `Series`, `displ_size`

In [8]:
scraped_cellphones = pd.read_csv(data_path+"cellphones_scraped.csv",index_col='url')
# cleaned_cellphones2 = pd.read_csv("Laptop-EDA/scraped_data/cellphones_cleaned.csv",index_col='url')
cleaned_cellphones = pd.read_csv(data_path+"cellphones-cleaned.csv",index_col='url')

cleaned_cellphones.drop(columns=['1star', '2star', '3star', '4star', '5star'],inplace=True)
cleaned_cellphones.rename(columns={'comment_count':'Comment_count',
},inplace=True)
cleaned_cellphones['Price'] = cleaned_cellphones['Price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_cellphones.info()
#     cleaned_cellphones.info()
#     cleaned_cellphones2.info()
#     # print(scraped_cellphones['Series'].value_counts(dropna=False))
# cleaned_cellphones.head()

for i,cleaned_row in cleaned_cellphones.Series.iteritems():
    r = scraped_cellphones.item_name[i]
    if not isinstance(r, str):
        # print(r)
        r = r[0]
    black_key = [r'\d+gb',r'\d+inch',r'\d+gpu',r'\d+tb',
                 'laptop','chính','hãng','việt','nam','nhập','khẩu','cũ','đẹp',
                 'apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft','huawei','avita','gigabyte']
    cpu_key = ['core','ryzen','m1']
    specs_key = [r'\d+\sgb',r'[\d.]+\sinch',r'\d+\sgpu',r'\d+\stb']
    extract_r1 = []
    for index,w in enumerate(r.lower().split()):
        if w in cpu_key:
            break
        if not any(bool(re.match(k,w)) for k in black_key) or (w == 'laptop' and index!=0):
            extract_r1.append(w)
    extract_r1 = ' '.join(extract_r1).strip(' -')
    for k in specs_key:
        extract_r1 = re.sub(k, '', extract_r1).strip(' -')

    # extract_r2 = re.search(r'([\w\s-]+)',r.lower())
    # print('{0:80} {1}'.format(r,extract_r1))
    print(f'{r:80} {cleaned_row:20} {extract_r1.strip()}')
    cleaned_cellphones.Series[i] = extract_r1.strip()

cleaned_cellphones.Rating.fillna(0,inplace=True)
#recleaned displ_size
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-asus-rog-strix-g17-g713qr-hg073t.html'] = 17.3
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-hp-15-dy2093dx-405f7ua.html'] = 15.6
#lower all string attributes
lower_all_string(cleaned_cellphones)
# cleaned_cellphones.head()

Apple MacBook Air 13 inch 128GB MQD32 I Chính hãng Apple Việt Nam                MacBook Air          macbook air  mqd32 i
Laptop Dell Inspiron 3505 Y1N1T2                                                 Inspiron             inspiron 3505 y1n1t2
Huawei Matebook 14                                                               Matebook             matebook 14
Huawei Matebook X                                                                Matebook X           matebook x
iMac 24 2021 M1 7GPU 8GB 256GB I Chính hãng Apple Việt Nam                       iMac                 imac 24
Apple iMac 24 2021 M1 7GPU 16GB 256GB Chính Hãng                                 iMac                 imac 24
iMac 24 2021 M1 8GPU 8GB 256GB I Chính hãng Apple Việt Nam                       iMac                 imac 24
iMac 24 2021 M1 8GPU 16GB 256GB I Chính hãng Apple Việt Nam                      iMac                 imac 24
iMac 24 2021 M1 8GPU 16GB 512GB I Chính hãng Apple Việt Nam                      iMac  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Laptop Dell Inspiron 14 5410 J42F81                                              Inspiron             inspiron 14 5410 j42f81
Laptop Dell Inspiron 14 5410 P143G001ASL                                         Inspiron             inspiron 14 5410 p143g001asl
Laptop Dell Inspiron 15 3502                                                     Inspiron             inspiron 15 3502
Laptop Dell Inspiron 15 3505 Y1N1T5                                              Inspiron             inspiron 15 3505 y1n1t5
Laptop Dell Inspiron 3511 26F1K                                                  Inspiron             inspiron 3511 26f1k
Laptop Dell Inspiron 3501 5081BLK                                                Inspiron             inspiron 3501 5081blk
Laptop Dell Inspiron 3501 5580BLK - Nhập khẩu chính hãng                         Inspiron             inspiron 3501 5580blk
Laptop Dell Inspiron 3511 5G8TF                                                  Inspiron             inspiron 3511 5g8tf
Lapto

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


## Laptop Arena


In [9]:
la_mapping_cols = ['Brand','Family','Series','Model name','Part Number',
                   'Processor manufacturer','Processor model','Internal memory',
                   'Processor series','Processor family','Release Date',
                   'Total storage capacity','Display diagonal','Maximum refresh rate']
def import_laptoparena(data_path):
    frames = []
    for file_path in glob.glob(data_path+'laptoparena*'):
        df = pd.read_csv(file_path,index_col='product_url')
        frames.append(df)
    df = pd.concat(frames)
    return df.reindex(sorted(df.columns),axis=1)

laptoparena_scrapped = import_laptoparena(data_path)
print(len(laptoparena_scrapped.columns))


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,9,12,15,16,22,27,31,38,40,43,44,49,56,58,61,63,65,67,69,70,73,74,75,76,77,80,85,86,87,89,91,96,103,106,110,111,113,114,123,126,135,136,142,143,144,147,153,155,164,167,168,170,172,174,176,180,181,187,188,190,194,196,199,204,207,208,214,216,219,220,225,230,231,237,239,242,249,250,252,255,256,258,260,261,262,266,269,271,272,273,274,276,278,288,289,295,300,308,311,324,328,334,341,345,346,347,351,357,358,364,370,371,375,376,377,378,379,381,383,387,393,397,398,399,402,404,405,408,409,410,411,416,417,419,421,425,427,429,433,434,437,438) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0,1,2,4,6,7,8,9,11,12,18,20,23,24,29,35,39,41,42,46,47,48,49,50,54,60,61,68,69,72,75,76,79,82,87,89,91,93,94,98,99,100,102,104,106,117,130,134,136,137,13

604


In [10]:
print('Width: ', len(laptoparena_scrapped.columns))
print('Length: ',len(laptoparena_scrapped))
for c in la_mapping_cols:
    text_facet(laptoparena_scrapped,c,print_all=False)

Width:  604
Length:  77464
---------- Brand ------------
DELL         15677
Acer         12097
HP           11833
Lenovo       11833
ASUS         11832
MSI           5115
Fujitsu       2092
Microsoft     1830
Toshiba       1362
Dynabook      1253
Alienware      565
Gigabyte       549
MEDION         454
Samsung        406
LG             380
Huawei         153
Sony            22
Thomson         11
Name: Brand, dtype: int64
Number of null: 0
---------- Family ------------
NaN                     7909
ThinkPad                6349
Latitude                5209
Aspire                  4630
Inspiron                3605
                        ... 
ZBook x2                   1
Style                      1
3000                       1
ImagineBook                1
Galaxy Book Flex2 5G       1
Name: Family, Length: 168, dtype: int64
Number of null: 7909
---------- Series ------------
NaN      18633
5000      5562
3         4185
3000      3625
15        3591
         ...  
1020         1
350       

In [11]:
def clean_laptoparena(source,use_part_number=True):
    result = source[source.Brand.str.lower().isin(all_brands)]
    result['Brand'] = source['Brand'].str.strip()
    
    result['Series'] = source['product_title'].str.split(' - ').str[0]
    result['product_title'] = source['Series']

    result['cpu_brand'] = source['Processor manufacturer'].str.strip().str.lower()
    source[['Processor family','Processor series']] = source[['Processor family','Processor series']].fillna('')
    for i,r in source.iterrows():
        if pd.isnull(r['Processor manufacturer']):
            row = ' '.join(r[['Processor family','Processor series']]).lower()
            for chip in ('intel','qualcomm','amd'):
                if chip in row:
                    result.cpu_brand[i] = chip

    # text_facet(source,'Processor model',print_all=False)
    result['cpu_code'] = source['Processor model'].str.split('-').str[-1].str.strip()
    # text_facet(result,'cpu_code',print_all=False)

    # text_facet(source,'Internal memory',print_all=False)
    result['ram_size'] = source['Internal memory'].str.extract(r'(\d+)',expand=False).str.strip().astype('float64')
    # text_facet(result,'ram_size',print_all=False)

    # text_facet(source,'Total storage capacity',print_all=False)
    result['storage_size'] = source['Total storage capacity'].str.extract(r'(\d+)',expand=False).str.strip().astype('float64')
    result['storage_size'][result['storage_size'] == 1000] = 1024
    # text_facet(result,'storage_size',print_all=False)

    # text_facet(source,'Display diagonal',print_all=False)
    result['displ_size'] = source['Display diagonal'].str.extract(r'([\d.]+)',expand=False).str.strip().astype('float64')
    # text_facet(result,'displ_size',print_all=False)

    # text_facet(source,'Maximum refresh rate',print_all=False)
    result['displ_rate'] = source['Maximum refresh rate'].str.extract(r'([\d.]+)',expand=False).str.strip().fillna(60).astype('int64')
    # text_facet(result,'displ_rate',print_all=False)
    lower_all_string(result)

    result.dropna(subset=mapping_cols+['Release Date'],inplace=True)
    for c in result.columns:
        if len(result[result[c].isnull()]) > len(result)/2:
            result.drop(columns=c,inplace=True)
            continue
        else:
            text_facet(result,c,print_all=False)
    return result

cleaned_laptoparena = clean_laptoparena(laptoparena_scrapped)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

---------- AC adapter power ------------
65 w      21347
45 w      19733
NaN       12422
180 w      3428
230 w      3142
135 w      2124
130 w      1856
150 w      1699
90 w       1388
120 w      1089
33 w        752
200 w       567
170 w       445
240 w       431
280 w       427
330 w       341
39 w        215
40 w        213
127 w       125
30 w        108
95 w         87
102 w        82
42 w         57
10 w         39
560 w        36
44 w         35
24 w         20
20 w         19
100 w        18
18 w         11
48 w         10
15 w         10
460 w         9
60 w          6
250 w         6
27 w          5
300 w         5
80 w          4
54 w          3
85 w          2
46 w          2
125 w         1
98 w          1
5 w           1
660 w         1
185 w         1
52 w          1
38 w          1
1000 w        1
2000 w        1
290 w         1
75 w          1
12 w          1
160 w         1
64 w          1
658 w         1
23 w          1
Name: AC adapter power, dtype: int64
Number of 

In [12]:
print(cleaned_laptoparena.cpu_code[cleaned_laptoparena.cpu_code.isnull()])

Series([], Name: cpu_code, dtype: object)


In [13]:
print(laptoparena_scrapped[['Processor manufacturer','Processor series','Processor family']][laptoparena_scrapped['Processor manufacturer'].isnull()])
# for c in .columns:
#     if 'processor' in c.lower():
#         text_facet(laptoparena,c)

                                          Processor manufacturer  ...        Processor family
product_url                                                       ...                        
https://www.laptoparena.net/product/20001                    NaN  ...   8th gen Intel Core i7
https://www.laptoparena.net/product/20002                    NaN  ...   9th gen Intel Core i7
https://www.laptoparena.net/product/20003                    NaN  ...   9th gen Intel Core i7
https://www.laptoparena.net/product/20004                    NaN  ...   9th gen Intel Core i7
https://www.laptoparena.net/product/20005                    NaN  ...   9th gen Intel Core i7
...                                                          ...  ...                     ...
https://www.laptoparena.net/product/72033                    NaN  ...  10th gen Intel Core i7
https://www.laptoparena.net/product/72034                    NaN  ...  10th gen Intel Core i5
https://www.laptoparena.net/product/72035                   

# Integrate
cleaned data are saved as `cleaned_tgdd`,`cleaned_fpt`,`cleaned_cellphones`

TODO
- Chọn các attribute và xử lý các attribute đó trong cả 3 file - Series ✅
- Kết nối 3 data của tgdd, cellphone và fpt bao gồm các mapping attribute và hợp nhất các retained attribute ✅ (cách hợp nhất các retained phải xem lại)
- Kết nối data thu được ở bước 2 với data của laptop arena dựa vào các mapping attribute, giữ lại các retained.
- Xử lý data đã map được ở bước 3 (loại bỏ các cột quá nhiều null, format dữ liệu, điền tay cho các máy của apple)


## Check out mapping attribute in TGDD, FPT and CellphoneS
- Mapping attribute must be non-null, except for Apple brand ✅
- Retained attribute type is float or int ✅
- Three dataset must have same set of column ✅
- No duplicated rows ✅

In [14]:
cleaned_fpt[['Series','Brand','cpu_brand','cpu_code','ram_size','storage_size',
             'displ_size','displ_rate',
             'Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_fpt.Brand[cleaned_fpt.cpu_code.isnull()])
    print(cleaned_fpt[cleaned_fpt['P/N'].isnull()])
cleaned_fpt.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series         146 non-null    object 
 1   Brand          146 non-null    object 
 2   cpu_brand      146 non-null    object 
 3   cpu_code       145 non-null    object 
 4   ram_size       146 non-null    int64  
 5   storage_size   146 non-null    int64  
 6   displ_size     146 non-null    float64
 7   displ_rate     146 non-null    int64  
 8   Price          146 non-null    int64  
 9   Rating         143 non-null    float64
 10  Rating_count   146 non-null    int64  
 11  Comment_count  143 non-null    float64
dtypes: float64(3), int64(5), object(4)
memory usage: 13.8+ KB
66    apple
Name: Brand, dtype: object
Empty DataFrame
Columns: [P/N, Rating, Rating_count, Comment_count, Brand, Series, Price, cpu_brand, cpu_code, ram_size, ram_upgradable, GPU, Battery, storage_type, storage_size

,P/N,Rating,Rating_count,Comment_count,Brand,Series,Price,cpu_brand,cpu_code,ram_size,ram_upgradable,GPU,Battery,storage_type,storage_size,storage_extra_slot,displ_size,displ_resolution,displ_rate,screen_technology,Weight,length,width,thickness,OS,Surface,Release_year,hdmi,headphone_jack,lan,type_c,usb_a,sd_card,backlit_keyboard
0,nh.qd9sv.001,0.0,0,598.0,acer,nitro gaming an515-57-74nu nh.qd9sv.001,28499000,intel,11800h,8,yes,nvidia geforce rtx 3050ti 4gb,--,ssd,512,yes,15.6,1920 x 1080,144,led-backlit,2.2,363.4,255,23.9,windows 10,nhựa,2021,yes,yes,yes,yes,yes,no,yes
1,852vn,0.0,0,340.0,msi,modern 14 b11mou 852vn,18799000,intel,1155g7,8,yes,intel iris xe graphics,65 w,ssd,512,yes,14.0,1920 x 1080,60,ips lcd,1.3,319,219,16.9,windows 10,kim loại,2021,yes,yes,yes,yes,yes,yes,yes
2,p112f001,0.0,0,116.0,dell,inspiron n3511 p112f001,15299000,intel,1115g4,4,yes,intel uhd graphics,65 w,ssd,256,yes,15.6,1920 x 1080,60,wva anti-glare led backlit narrow border,1.73,358.5,258,18.9,windows 10,nhựa,2021,yes,yes,yes,yes,yes,yes,yes


In [15]:
cleaned_cellphones[['P/N','Series','Brand','cpu_brand','cpu_code','ram_size','storage_size','displ_size','displ_rate','Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_cellphones.Brand[cleaned_cellphones.cpu_code.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.ram_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.storage_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.displ_size.isnull()])
cleaned_cellphones.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 423 entries, https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html to https://cellphones.com.vn/surface-pro-7-plus.html
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   P/N            357 non-null    object 
 1   Series         423 non-null    object 
 2   Brand          423 non-null    object 
 3   cpu_brand      423 non-null    object 
 4   cpu_code       401 non-null    object 
 5   ram_size       422 non-null    float64
 6   storage_size   422 non-null    float64
 7   displ_size     418 non-null    float64
 8   displ_rate     423 non-null    int64  
 9   Price          357 non-null    float64
 10  Rating         423 non-null    float64
 11  Rating_count   423 non-null    int64  
 12  Comment_count  423 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 66.3+ KB
url
https://cellphones.com.vn/apple-macbook-air-13-i

,sd_card,P/N,ram_upgradable,GPU,displ_rate,backlit_keyboard,Weight,thickness,width,length,screen_technology,Battery,OS,displ_resolution,Surface,displ_size,Price,Comment_count,Rating_count,Rating,ram_size,cpu_code,cpu_brand,storage_size,storage_type,storage_extra_slot,Release_year,Brand,Series,usb_a,type_c,lan,headphone_jack,hdmi
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html,yes,mqd32,NaN,intel hd 6000,60,NaN,1.35,17,227.0,325.00,NaN,54.0,macos,1440 x 900,NaN,13.3,20000000.0,306,0,0.0,8.0,NaN,intel,128.0,ssd,NaN,2017.0,apple,macbook air mqd32 i,yes,no,no,no,no
https://cellphones.com.vn/aptop-dell-inspiron-15-3505-y1n1t2.html,no,y1n1t2,yes,amd radeon,60,no,1.83,18,249.0,363.96,NaN,42.0,windows 10,1920 x 1080,NaN,15.6,18290000.0,28,0,0.0,8.0,3500u,amd,512.0,ssd,NaN,NaN,dell,inspiron 3505 y1n1t2,yes,no,yes,yes,yes
https://cellphones.com.vn/huawei-matebook-14.html,no,NaN,NaN,nvidia geforce mx350,60,NaN,1.53,15.9,223.8,307.50,NaN,56.0,windows 10,2160 x 1440,NaN,14.0,NaN,3,0,0.0,8.0,10210u,intel,512.0,ssd,NaN,NaN,huawei,matebook 14,no,yes,no,yes,yes


In [16]:
cleaned_tgdd[['Series','Brand','cpu_brand','cpu_code','ram_size','storage_size','displ_size','displ_rate','Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_tgdd.Brand[cleaned_tgdd.cpu_code.isnull()])
cleaned_tgdd.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 181 entries, https://www.thegioididong.com/laptop/msi-gf66-11uc-i7-224vn?src=osp#top-tskt to https://www.thegioididong.com/laptop/acer-aspire-a315-56-308n-i3-nxhs5sv00c?src=osp#top-tskt
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series         181 non-null    object 
 1   Brand          181 non-null    object 
 2   cpu_brand      181 non-null    object 
 3   cpu_code       174 non-null    object 
 4   ram_size       181 non-null    float64
 5   storage_size   181 non-null    float64
 6   displ_size     181 non-null    float64
 7   displ_rate     181 non-null    int64  
 8   Price          181 non-null    float64
 9   Rating         181 non-null    float64
 10  Rating_count   181 non-null    int64  
 11  Comment_count  181 non-null    int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 23.4+ KB
url
https://www.thegioididong.com/laptop/apple-macbook-pr

,P/N,Rating,Rating_count,Comment_count,Brand,Series,Price,cpu_brand,cpu_code,ram_size,ram_upgradable,GPU,Battery,storage_size,storage_type,storage_extra_slot,displ_size,displ_resolution,displ_rate,screen_technology,Weight,length,width,thickness,OS,Surface,Release_year,backlit_keyboard,hdmi,headphone_jack,lan,type_c,usb_a,sd_card
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://www.thegioididong.com/laptop/msi-gf66-11uc-i7-224vn?src=osp#top-tskt,224vn,3.0,5,70,msi,katana gaming gf66 11uc 224vn,29990000.0,intel,11800h,8.0,yes,nvidia geforce rtx3050,53.0,512.0,ssd,no,15.6,1920 x 1080,144,ips,2.10,359.0,259.0,24.90,windows 10,plastic,2021.0,yes,yes,yes,yes,yes,yes,no
https://www.thegioididong.com/laptop/msi-gaming-pulse-gl66-11udk-i7-816vn?src=osp#top-tskt,816vn,0.0,0,13,msi,gaming pulse gl66 11udk 816vn,33990000.0,intel,11800h,16.0,yes,nvidia geforce rtx3050ti,53.5,512.0,ssd,no,15.6,1920 x 1080,144,ips,2.25,359.0,259.0,23.95,windows 10,metal,2021.0,yes,yes,no,yes,yes,yes,no
https://www.thegioididong.com/laptop/acer-nitro-gaming-an515-57-727j-i7-nhqd9sv005?src=osp#top-tskt,nh.qd9sv.005,3.6,13,220,acer,nitro 5 gaming an515 57 727j nh.qd9sv.005,29990000.0,intel,11800h,8.0,yes,nvidia geforce rtx3050ti,57.0,512.0,ssd,yes,15.6,1920 x 1080,144,ips,2.20,363.4,255.0,23.90,windows 10,plastic,2021.0,yes,yes,yes,yes,yes,yes,no


In [17]:
print(sorted(cleaned_fpt.columns))
print(sorted(cleaned_tgdd.columns))
print(sorted(cleaned_cellphones.columns))

['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Weight', 'backlit_keyboard', 'cpu_brand', 'cpu_code', 'displ_rate', 'displ_resolution', 'displ_size', 'hdmi', 'headphone_jack', 'lan', 'length', 'ram_size', 'ram_upgradable', 'screen_technology', 'sd_card', 'storage_extra_slot', 'storage_size', 'storage_type', 'thickness', 'type_c', 'usb_a', 'width']
['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Weight', 'backlit_keyboard', 'cpu_brand', 'cpu_code', 'displ_rate', 'displ_resolution', 'displ_size', 'hdmi', 'headphone_jack', 'lan', 'length', 'ram_size', 'ram_upgradable', 'screen_technology', 'sd_card', 'storage_extra_slot', 'storage_size', 'storage_type', 'thickness', 'type_c', 'usb_a', 'width']
['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Wei

In [18]:
print(f"Duplicated tgdd: {any(cleaned_tgdd.duplicated())}")
print(f"Duplicated cellphones: {any(cleaned_cellphones.duplicated())}")
print(f"Duplicated fpt: {any(cleaned_fpt.duplicated())}")

Duplicated tgdd: False
Duplicated cellphones: False
Duplicated fpt: False


Rating_count chia làm 2 tập

In [19]:
# col = 'Rating'
# col = 'Release_year'
# col = 'Comment_count'
# col = 'Brand'
# col = 'storage_type'
for col in mapping_cols:
    for data in (cleaned_cellphones,cleaned_fpt,cleaned_tgdd):
        text_facet(data[data.Brand == 'apple'],col)
# data = cleaned_tgdd
# data[data.Brand == 'apple'].Series

---------- Series ------------
imac 24                                             6
macbook air                                         4
mac mini                                            4
macbook pro 13 touch bar                            4
macbook pro                                         3
macbook pro 13 touch bar i5 1.4 i                   2
macbook pro 14                                      1
macbook pro 13 touch bar i5 2.0 i                   1
imac pro                                            1
imac 21.5 i5 2.3 (mhk03)                            1
imac 27 5k i7 3.8 radeon 5500xt (mxwv2)             1
imac 27 5k i5 3.3 radeon 5300 (mxwu2)               1
imac 27 5k i5 3.1 radeon 5300 (mxwt2)               1
imac 21.5 4k i5 3.0 radeon 560x (mhk33)             1
macbook pro 16 touch bar i7 2.6 2019 bạc (mvvl2)    1
macbook pro 16                                      1
macbook air  mqd32 i                                1
Name: Series, dtype: int64
---------- Series ------

## Merge TGDD, FPT and CellphoneS
Retained attributes:
- Comment = Sum of 3
- Rating Count = Sum of 3
- Availabe_in = Number of existence in stores
- Rating point = Average of `Rating` with weight is `Rating_count`
- Price = Average of `Price`

String similarity
<ol>
<li><a href="http://en.wikipedia.org/wiki/Hamming_distance">Hamming distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Levenshtein_distance">Levenshtein distance</a></li>
<li><a href="http://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance">Damerau–Levenshtein distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance">Jaro–Winkler distance</a></li>
</ol>

Jellyfish: https://github.com/jamesturk/jellyfish

In [20]:
def similar_str(a, b, algo):
    '''
    Compare string similarity with threshold
    Input: a,b - string, thres - float 
    Output: float
    '''
    if algo == 'normal':
        return SequenceMatcher(None, a, b).ratio()
    elif algo == 'lev':
        return fuzz.ratio(a,b)/100

s1 = 'gaming predator helios 300 ph315-54-78w5 nh.qc5sv.001'
s2 = 'Predator Helios 300 PH315-53-781R NH.Q7YAA.001'.lower()
s3 = 'Acer Nitro AN515-51-79DZ - NH.Q2QAA.007'.lower()
s4 = 'nitro 5 an515 57 74nu nh.qd9sv.001'
for i in ('Nitro','5','AN515-51-79DZ','NH.Q2QAA.007'):
    print(similar_str(i.lower(),s4,'lev'))
# print(similar_str(s2,s1,'lev'))
print(similar_str(s3,s4,'lev'))

0.26
0.06
0.34
0.3
0.6


In [21]:
def merge(dfa,dfb,pref1='_df1',pref2='_df2',laptoparena_position=None):
    min_score = 0.55
    algo = 'lev'

    df1 = dfa.copy()
    df2 = dfb.copy()

    df1['Index'+pref1]=df1.index
    df2['Index'+pref2]=df2.index

    df1['match_index'+pref2] = np.nan
    df1['match_score'+pref2] = np.nan
    
    choices = []
    for i1,r1 in df1.iterrows():
        for i2,r2 in df2.iterrows():
            score = similar_str(r1['Series'],r2['Series'],algo)
            condition = [r1['ram_size'] == r2['ram_size'],
                         r1['cpu_brand'] == r2['cpu_brand'],
                         r1['cpu_code'] == r2['cpu_code'],
                         r1['Brand'] == r2['Brand'],
                         r1['storage_size'] == r2['storage_size'],
                         r1['displ_rate'] == r2['displ_rate'],
                         abs(r1['displ_size']-r2['displ_size'])<0.5,
                         score > min_score
            ]
            if all(condition):
                choices.append([i1,i2,score])

    exclude1, exclude2 = [], []
    for i1,i2,score in sorted(choices,key=lambda x:x[2],reverse=True):
        if i1 not in exclude1 and i2 not in exclude2:
            df1['match_index'+pref2][i1] = i2
            df1['match_score'+pref2][i1] = score
            exclude1.append(i1)
            exclude2.append(i2)
    
    print(f"Matched: {len(df1['match_index'+pref2][np.invert(df1['match_index'+pref2].isnull())])}")

    merged = pd.merge(df1, df2, suffixes=[pref1,pref2], 
                      left_on='match_index'+pref2, right_index = True, 
                      how = laptoparena_position if bool(laptoparena_position) else 'outer'
                      )
    merged.reset_index(inplace=True)
    return merged.reindex(sorted(merged.columns), axis=1)

def join_column_from_merged(df, pref1, pref2,priority='left',keep_cols=[]):
    merged_retained = []
    for c in df.columns:
        # print(c)
        if any(k+'_' in c for k in retained_cols + keep_cols):
            merged_retained.append(c)
    print(merged_retained)
    result = df[merged_retained]

    # to_merge = mapping_cols + other_cols
    to_merge = all_cols
    for c in to_merge:
        result[c] = np.nan
        for i,r in result.iterrows():
            if pd.isnull(df[c+pref1][i]):
                result[c][i] = df[c+pref2][i]
            elif pd.isnull(df[c+pref2][i]):
                result[c][i] = df[c+pref1][i]
            elif priority == 'left':
                result[c][i] = df[c+pref1][i]
            elif priority == 'right':
                result[c][i] = df[c+pref2][i]
    return result

def join_retained_atrr(dataframe, prefs):
    df = dataframe.copy()
    df['Availabe_in'] = len(prefs) - df[['Index'+p for p in prefs]].isnull().sum(axis=1)
    for i,r in df.iterrows():
        r.Rating_count = np.sum(r[['Rating_count'+p for p in prefs]])
        if r.Rating_count ==0:
            r.Rating = 0
        else:
            r.Rating = np.average(r[['Rating'+p for p in prefs]].fillna(0),
                                weights = r[['Rating_count'+p for p in prefs]].fillna(0))
        r.Comment_count = np.sum(r[['Comment_count'+p for p in prefs]])
        r.Price = np.mean(r[['Price'+p for p in prefs]])
        df.loc[i] = r

    for c in retained_cols:
        with pd.option_context('display.max_columns', None):
            print(df[[c]+[c+p for p in prefs]])

    for c in df.columns:
        if any(k+'_' in c for k in retained_cols + ['Index'+p for p in prefs]):
            df = df.drop(columns=c)
    return df

In [22]:
df = merge(cleaned_fpt, cleaned_tgdd,'_fpt','_tgdd')
# df = merge(cleaned_tgdd, cleaned_fpt,'_tgdd','_fpt')
df_2 = join_column_from_merged(df,'_fpt','_tgdd',priority='right',
                               keep_cols=['Index','Series','match_score'])
df_3 = merge(df_2,cleaned_cellphones,'_fpt_tgdd','_cellphones')
df_4 = join_column_from_merged(df_3,'_fpt_tgdd','_cellphones',priority='left',keep_cols=['Index'])
df_4.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59:

Matched: 61
['Comment_count_fpt', 'Comment_count_tgdd', 'Index_fpt', 'Index_tgdd', 'Price_fpt', 'Price_tgdd', 'Rating_count_fpt', 'Rating_count_tgdd', 'Rating_fpt', 'Rating_tgdd', 'Series_fpt', 'Series_tgdd', 'match_score_tgdd']


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[key] = value


Matched: 121
['Comment_count_cellphones', 'Comment_count_fpt', 'Comment_count_fpt_tgdd', 'Comment_count_tgdd', 'Index_cellphones', 'Index_fpt', 'Index_fpt_tgdd', 'Index_tgdd', 'Price_cellphones', 'Price_fpt', 'Price_fpt_tgdd', 'Price_tgdd', 'Rating_cellphones', 'Rating_count_cellphones', 'Rating_count_fpt', 'Rating_count_fpt_tgdd', 'Rating_count_tgdd', 'Rating_fpt', 'Rating_fpt_tgdd', 'Rating_tgdd']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Comment_count_cellphones,Comment_count_fpt,Comment_count_fpt_tgdd,Comment_count_tgdd,Index_cellphones,Index_fpt,Index_fpt_tgdd,Index_tgdd,Price_cellphones,Price_fpt,Price_fpt_tgdd,Price_tgdd,Rating_cellphones,Rating_count_cellphones,Rating_count_fpt,Rating_count_fpt_tgdd,Rating_count_tgdd,Rating_fpt,Rating_fpt_tgdd,Rating_tgdd,Comment_count,Price,Rating,Rating_count,Series,Brand,cpu_brand,cpu_code,ram_size,storage_size,displ_size,displ_rate,Battery,GPU,OS,P/N,displ_resolution,hdmi,Release_year,Surface,Weight,backlit_keyboard,lan,length,ram_upgradable,screen_technology,sd_card,storage_extra_slot,storage_type,headphone_jack,thickness,type_c,usb_a,width
0,44.0,598.0,264.0,264.0,https://cellphones.com.vn/laptop-gaming-acer-n...,0.0,0.0,https://www.thegioididong.com/laptop/acer-nitr...,27390000.0,28499000.0,29990000.0,29990000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,264.0,29990000.0,0.0,0.0,nitro 5 gaming an515 57 74nu nh.qd9sv.001,acer,intel,11800h,8.0,512.0,15.6,144.0,57,nvidia geforce rtx3050ti,windows 10,nh.qd9sv.001,1920 x 1080,yes,2021,plastic,2.2,yes,yes,363.4,yes,ips,no,yes,ssd,yes,23.9,yes,yes,255
1,7.0,340.0,53.0,53.0,https://cellphones.com.vn/laptop-msi-modern-14...,1.0,1.0,https://www.thegioididong.com/laptop/msi-gamin...,21990000.0,18799000.0,22990000.0,22990000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,22990000.0,0.0,0.0,gaming modern 14 b11sbu 669vn,msi,intel,1155g7,8.0,512.0,14.0,60.0,39,nvidia geforce mx450,windows 10,669vn,1920 x 1080,yes,2021,metal,1.3,yes,no,319,yes,ips,yes,no,ssd,yes,16.9,yes,yes,219
2,NaN,116.0,206.0,206.0,NaN,2.0,2.0,https://www.thegioididong.com/laptop/dell-insp...,NaN,15299000.0,15290000.0,15290000.0,NaN,NaN,0.0,18.0,18.0,0.0,3.7,3.7,206.0,15290000.0,3.7,18.0,inspiron 15 3511 p112f001abl,dell,intel,1115g4,4.0,256.0,15.6,60.0,41,intel uhd graphics,windows 10,p112f001abl,1920 x 1080,yes,2021,plastic,1.7,no,no,358.5,yes,va,yes,yes,ssd,yes,18.9,no,yes,235.5
3,NaN,378.0,378.0,NaN,NaN,4.0,4.0,NaN,NaN,14999000.0,14999000.0,NaN,NaN,NaN,4.0,4.0,NaN,5.0,5.0,NaN,378.0,14999000.0,5.0,4.0,ideapad slim 3 15itl6 82h80040vn,lenovo,intel,1115g4,8.0,512.0,15.6,60.0,--,intel uhd graphics,windows 10,82h80040vn,1920 x 1080,yes,2021,abs plastic,1.65,yes,yes,359,yes,anti-glare led-backlit,yes,yes,ssd,yes,19.9,yes,yes,237.7
4,NaN,1244.0,1244.0,NaN,NaN,6.0,5.0,NaN,NaN,18799000.0,18799000.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,1244.0,18799000.0,5.0,5.0,ideapad slim 5 15itl05 82fg00r0vn,lenovo,intel,1135g7,16.0,512.0,15.6,60.0,45 w,intel iris xe graphics,windows 10,82fg00r0vn,1920 x 1080,yes,2021,kim loại và nhựa,1.66,yes,yes,356.67,yes,"ips lcd led backlit, true tone",yes,yes,ssd,yes,17.9,yes,yes,233.13


In [23]:
df_5 = join_retained_atrr(df_4,('_fpt','_tgdd','_cellphones'))
df_5.head()

     Comment_count  Comment_count_fpt  Comment_count_tgdd  \
0            906.0              598.0               264.0   
1            400.0              340.0                53.0   
2            322.0              116.0               206.0   
3            378.0              378.0                 NaN   
4           1244.0             1244.0                 NaN   
..             ...                ...                 ...   
563           39.0                NaN                 NaN   
564           21.0                NaN                 NaN   
565           76.0                NaN                 NaN   
566           49.0                NaN                 NaN   
567           29.0                NaN                 NaN   

     Comment_count_cellphones  
0                        44.0  
1                         7.0  
2                         NaN  
3                         NaN  
4                         NaN  
..                        ...  
563                      39.0  
564        

,Index_cellphones,Index_fpt,Index_tgdd,Comment_count,Price,Rating,Series,Brand,cpu_brand,cpu_code,ram_size,storage_size,displ_size,displ_rate,Battery,GPU,OS,P/N,displ_resolution,hdmi,Release_year,Surface,Weight,backlit_keyboard,lan,length,ram_upgradable,screen_technology,sd_card,storage_extra_slot,storage_type,headphone_jack,thickness,type_c,usb_a,width,Availabe_in
0,https://cellphones.com.vn/laptop-gaming-acer-n...,0.0,https://www.thegioididong.com/laptop/acer-nitr...,906.0,2.862633e+07,0.0,nitro 5 gaming an515 57 74nu nh.qd9sv.001,acer,intel,11800h,8.0,512.0,15.6,144.0,57,nvidia geforce rtx3050ti,windows 10,nh.qd9sv.001,1920 x 1080,yes,2021,plastic,2.2,yes,yes,363.4,yes,ips,no,yes,ssd,yes,23.9,yes,yes,255,3
1,https://cellphones.com.vn/laptop-msi-modern-14...,1.0,https://www.thegioididong.com/laptop/msi-gamin...,400.0,2.125967e+07,0.0,gaming modern 14 b11sbu 669vn,msi,intel,1155g7,8.0,512.0,14.0,60.0,39,nvidia geforce mx450,windows 10,669vn,1920 x 1080,yes,2021,metal,1.3,yes,no,319,yes,ips,yes,no,ssd,yes,16.9,yes,yes,219,3
2,NaN,2.0,https://www.thegioididong.com/laptop/dell-insp...,322.0,1.529450e+07,3.7,inspiron 15 3511 p112f001abl,dell,intel,1115g4,4.0,256.0,15.6,60.0,41,intel uhd graphics,windows 10,p112f001abl,1920 x 1080,yes,2021,plastic,1.7,no,no,358.5,yes,va,yes,yes,ssd,yes,18.9,no,yes,235.5,2
3,NaN,4.0,NaN,378.0,1.499900e+07,5.0,ideapad slim 3 15itl6 82h80040vn,lenovo,intel,1115g4,8.0,512.0,15.6,60.0,--,intel uhd graphics,windows 10,82h80040vn,1920 x 1080,yes,2021,abs plastic,1.65,yes,yes,359,yes,anti-glare led-backlit,yes,yes,ssd,yes,19.9,yes,yes,237.7,1
4,NaN,6.0,NaN,1244.0,1.879900e+07,5.0,ideapad slim 5 15itl05 82fg00r0vn,lenovo,intel,1135g7,16.0,512.0,15.6,60.0,45 w,intel iris xe graphics,windows 10,82fg00r0vn,1920 x 1080,yes,2021,kim loại và nhựa,1.66,yes,yes,356.67,yes,"ips lcd led backlit, true tone",yes,yes,ssd,yes,17.9,yes,yes,233.13,1


In [24]:
def print_merged_result(df, col, prefs=('_fpt','_tgdd','_cellphones'),sort=True):
    '''prefs can be any length'''
    print(f"-------Column: {col} ---------------")
    if sort:
        df = df.iloc[df[[col+p for p in prefs]].fillna('').agg(''.join, axis=1).argsort()]
    for i, r in df.iterrows():
        # if abs(r[col+pref1]-r[col+pref2]) >=0.5:
        for p in prefs[1:]:
            print(f"{r['match_score'+p]:4} |",end=" ")
        for p in prefs:
            print(f"{r[col+p]:45} |",end=" ")
        for p in prefs:
            print(f"{r['Index'+p]:80} |",end=" ")
        print("")
# {r[col+pref1]:45} | {r[col+pref2]:40} | {r[col+pref3]:40} | {r['Index'+pref1]:10} | {r['Index'+pref2]:80} | {r['Index'+pref3]:80}")


# print_merged_result(df,col='Series')
print_merged_result(df_3,col='Series',sort=True)


-------Column: Series ---------------
 nan |  nan |                                           nan |                                           nan | 14-cf2033wm 3v7g4ua                           |                                                                              nan |                                                                              nan | https://cellphones.com.vn/laptop-hp-14-cf2033wm-3v7g4ua-pent.html                | 
 nan |  nan |                                           nan |                                           nan | 14-dq2055wm 39k15ua                           |                                                                              nan |                                                                              nan | https://cellphones.com.vn/laptop-hp-14-dq2055wm-39k15ua.html                     | 
 nan |  nan |                                           nan |                                           nan | 14s-dq2544tu 46m22pa                

In [25]:
r1 = cleaned_fpt.iloc[4]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(r1)

P/N                                         82h80040vn
Rating                                               5
Rating_count                                         4
Comment_count                                      378
Brand                                           lenovo
Series                ideapad slim 3 15itl6 82h80040vn
Price                                         14999000
cpu_brand                                        intel
cpu_code                                        1115g4
ram_size                                             8
ram_upgradable                                     yes
GPU                                 intel uhd graphics
Battery                                             --
storage_type                                       ssd
storage_size                                       512
storage_extra_slot                                 yes
displ_size                                        15.6
displ_resolution                           1920 x 1080
displ_rate

## Join merged result with LaptopArena
Differences when merging with LaptopArena compared to previous part
- Condition of `Series` will be different
- df1 and df2 has different set of columns
- Mapping column may contains `NaN`


In [ ]:
df_a = merge(df_5,cleaned_laptoparena,pref1='_vnstore',pref2='_laptoparena',laptoparena_position='left')

In [ ]:
df_a.head()